In [16]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent, PositionsTracker
from qubx.pandaz.utils import *
import qubx.pandaz.ta as pta

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.ta.indicators import sma, ema
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.utils.charting.lookinglass import LookingGlass

from qubx.backtester.simulator import simulate
from qubx.core.metrics import tearsheet
from qubx.trackers.sizers import FixedSizer, FixedRiskSizer
from qubx.core.metrics import chart_signals

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
r = CsvStorageDataReader("../tests/data/csv")

In [36]:
instr = lookup.find_symbol("BINANCE.UM", "BTCUSDT")
ohlc = r.read("BTCUSDT_ohlcv_M1", start='2024-01-01', stop='2024-01-15', transform=AsOhlcvSeries())
stream = r.read("BTCUSDT_ohlcv_M1", start='2024-01-01', stop='2024-01-15', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick))

In [37]:
stream[0].mid_price()

42314.0

In [47]:
LookingGlass([ohlc.pd()]).look(
    ).hline(stream[0].mid_price() + 1000
    ).hline(stream[0].mid_price() - 150
    ).hline(stream[0].mid_price() + 2000
    ).hline(stream[0].mid_price() - 1900
            ).hover()
# plt.axhline(stream[0].mid_price() + 150)

FigureWidget({
    'data': [{'close': array([42331.9, 42350.4, 42360.2, ..., 41814.3, 41734.9, 41763.1]),
              'high': array([42335.8, 42353.1, 42370.8, ..., 41853. , 41821.9, 41797.5]),
              'line': {'width': 1},
              'low': array([42289.6, 42331.8, 42349.6, ..., 41814.3, 41731.2, 41720. ]),
              'name': 'Master_0',
              'open': array([42314. , 42331.9, 42350.4, ..., 41848.7, 41814.3, 41734.9]),
              'type': 'candlestick',
              'uid': 'ff7ef48d-73f6-4f1c-8ac3-05b933bcc3d5',
              'x': array([datetime.datetime(2024, 1, 1, 0, 0),
                          datetime.datetime(2024, 1, 1, 0, 1),
                          datetime.datetime(2024, 1, 1, 0, 2), ...,
                          datetime.datetime(2024, 1, 14, 23, 58),
                          datetime.datetime(2024, 1, 14, 23, 59),
                          datetime.datetime(2024, 1, 15, 0, 0)], dtype=object),
              'xaxis': 'x',
              'yaxis': 